In [ ]:
!gcloud dataproc clusters list --region us-central1

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
!ls -l /usr/lib/spark/jars/graph*

# Imports

In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import math
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Spark

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf,SparkFiles
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
spark

# GCP (small corpus from hw3)


In [ ]:
# bucket_name = '206201667_316399773_bucket'
bucket_name = 'ir-engine-316399773'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.endswith('.parquet'):
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)

In [ ]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
# Import MultiFile and InvertedIndex classes
from inverted_index_gcp import *

In [ ]:
# take the 'text' and 'id' and create an RDD from it
doc_text_pairs = parquetFile.select("id", "text").rdd

# take the 'title' and 'id' and create an RDD from it
doc_title_pairs = parquetFile.select("id", "title").rdd

# take the 'anchor_text' and 'id' and create an RDD from it
doc_anchor_pairs = parquetFile.select("id", "anchor_text").rdd

In [ ]:
# When we look at the anchor text part, at the next stage we would want to count the terms' frequencies 
# according to the documents they are pointing to.

pointed_doc_anchor_pairs = doc_anchor_pairs.flatMap(
    lambda anchor_pair: anchor_pair[1]).groupByKey().mapValues(list).map(
        lambda pointed_anchor_pair: (
            pointed_anchor_pair[0], " ".join([word for word in pointed_anchor_pair[1]])))


In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)


In [ ]:
def tokenize(text, without_stopwords):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    if without_stopwords:
      tokens = [tok for tok in tokens if tok not in all_stopwords]
    return tokens

In [ ]:
def word_count(doc_part, id, without_stopwords):
  ''' Count the frequency of each word in a document part (text/title/anchor text) (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    doc_part: str
      Text of part of one document
    id: int
      Document id 
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = tokenize(doc_part, without_stopwords)
  token_counter = Counter(tokens)
  tokens_no_dups = []
  for token in tokens:
      if token not in tokens_no_dups:
        tokens_no_dups.append(token)
  return [(token, (id, token_counter[token])) for token in tokens_no_dups]

In [ ]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl, key=lambda tup: tup[0])

In [ ]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  # len(pl[1]) calculates df because in a posting list we have a list of documents the token is in.
  return postings.map(lambda pl: (pl[0], len(pl[1])))

In [ ]:
def calculate_doclen(doc_id, doc_part, without_stopwords):
  ''' For each document part (text/title-anchor) calculate its length (number of tokens).
  Parameters:
  -----------
    doc_part: str
      Text of part of one document (text/title/anchor)
    id: int
      Document id 
 Returns:
  --------
   Tuple
      (token, doclen) pair
  '''
  tokens = tokenize(doc_part,without_stopwords)
  return (doc_id, len(tokens))

In [ ]:
NUM_BUCKETS = 124   # todo: check about different number of buckets for GCP
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS


def partition_postings_and_write(postings, storage_path):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    offset: int
      The bucket number to start writing from.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  pl_to_buckets = postings.map(lambda pl: (token2bucket_id(pl[0]), (pl[0], pl[1])))
  buckets_val_one_list = pl_to_buckets.groupByKey().mapValues(list)
  buckets_locs = buckets_val_one_list.map(lambda bucket_pl: InvertedIndex.write_a_posting_list(bucket_pl, bucket_name, storage_path))
  return buckets_locs



In [ ]:
threshold = 50
idx_id = 0
idx_value = 1

# build index for Title:

In [ ]:
word_counts_title = doc_title_pairs.flatMap(lambda pair: word_count(pair[1], pair[0], False))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
#postings_title_filtered = postings_title.filter(lambda x: len(x[1])> threshold)
w2df_title = calculate_df(postings_title)
w2df_title_dict = w2df_title.collectAsMap()
doclen_title = doc_title_pairs.map(lambda pair: calculate_doclen(pair[idx_id], pair[idx_value], False)).collectAsMap()
posting_locs_list_title = partition_postings_and_write(postings_title, "index_title").collect()

# build index for Body:

In [ ]:
word_counts_text = doc_text_pairs.flatMap(lambda pair: word_count(pair[1], pair[0], True))
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_text_filtered = postings_text.filter(lambda x: len(x[1])> threshold)
w2df_text = calculate_df(postings_text_filtered)
w2df_text_dict = w2df_text.collectAsMap()
doclen_text = doc_text_pairs.map(lambda pair: calculate_doclen(pair[idx_id], pair[idx_value], True)).collectAsMap()
posting_locs_list_text = partition_postings_and_write(postings_text_filtered, "index_body").collect()

# build index for Anchor:

In [ ]:
word_counts_anchor = pointed_doc_anchor_pairs.flatMap(lambda pair: word_count(pair[1], pair[0], False))
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)
#postings_anchor.filter(lambda x: len(x[1])> threshold)
w2df_anchor = calculate_df(postings_anchor)
w2df_anchor_dict = w2df_anchor.collectAsMap()
doclen_anchor = pointed_doc_anchor_pairs.map(lambda pair: calculate_doclen(pair[idx_id], pair[idx_value], True)).collectAsMap()
posting_locs_list_anchor = partition_postings_and_write(postings_anchor, "index_anchor").collect()


In [ ]:
def build_super_posting_loc(posting_locs_list, prefix):
  # merge the posting locations into a single dict
  super_posting_locs_title = defaultdict(list)
  for blob in client.list_blobs(bucket_name, prefix=prefix):
    if not blob.name.endswith("pickle"):
      continue
    with blob.open("rb") as f:
      posting_locs = pickle.load(f)
      for k, v in posting_locs.items():
        super_posting_locs_title[k].extend(v)
  return super_posting_locs_title

In [ ]:
super_posting_locs_text = build_super_posting_loc(posting_locs_list_text, 'postings_gcp/index_body')

In [ ]:
super_posting_locs_title = build_super_posting_loc(posting_locs_list_title, 'postings_gcp/index_title')

In [ ]:
super_posting_locs_anchor = build_super_posting_loc(posting_locs_list_anchor, 'postings_gcp/index_anchor')


In [ ]:
def create_tokens_counter_for_one_doc(curr_pair, without_stopwords):
      part_to_token = curr_pair[1]
      tokens = tokenize(part_to_token, without_stopwords)
      counter_of_tokens = Counter(tokens)
      doc_id = curr_pair[0]
      return (doc_id, counter_of_tokens)


In [ ]:
def build_index_by_part(part):
    '''Putting everything together (posting locations, df) and test that the resulting index is correct.'''
    inv_index = InvertedIndex()  # Create inverted index instance
    # Adding the posting locations dictionary to the inverted index
    # Add the token - df dictionary to the inverted index
    # add doclen field to inverted index instance
    
    # Add dictionary of (doc_id, title) pairs as a "connector" between text parts and title parts of documents.
    inv_index.doc_id_title = doc_title_pairs.collectAsMap()
    #inv_index.voc_size = len(inv_index.term_total)

    path_name = ""
    if part == 'text':  
        inv_index.posting_locs = super_posting_locs_text
        inv_index.df = w2df_text_dict
        inv_index.doclen = doclen_text
        doc_id_to_token_counter_text = doc_text_pairs.map(lambda pair: create_tokens_counter_for_one_doc(pair, True))

        inv_index.doc_id_2_norm = doc_id_to_norm_text
        path_name = 'index_body'
        inv_index.write_index('.', 'index_body')

    elif part == 'title':
        inv_index.posting_locs = super_posting_locs_title
        inv_index.df = w2df_title_dict
        inv_index.doclen = doclen_title
        n_docs = len(doclen_title)
        doc_id_to_token_counter_title = doc_title_pairs.map(lambda pair: create_tokens_counter_for_one_doc(pair, True))
        doc_id_to_norm_title = doc_id_to_token_counter_title.map(lambda pair: (pair[0],np.linalg.norm([(pair[1][term]/ doclen_title[pair[0]]) * math.log(n_docs/w2df_title_dict.get(term, n_docs),10)for term in pair[1]]))).collectAsMap()
        inv_index.doc_id_2_norm = doc_id_to_norm_title
        path_name = 'index_title'
        inv_index.write_index('.', 'index_title')

    elif part == 'anchor':
        inv_index.posting_locs = super_posting_locs_anchor
        inv_index.df = w2df_anchor_dict
        inv_index.doclen = doclen_anchor
        doc_id_to_token_counter_anchor = pointed_doc_anchor_pairs.map(lambda pair: create_tokens_counter_for_one_doc(pair, True))
        # exeception divided by zero -> need to remove anchor with no len (dont have pls for this document)
        not_zero = lambda anchor: inv_index.doclen[anchor[0]] != 0
        n_docs = len(doclen_anchor)
        docid_2_token_counter_without0 = doc_id_to_token_counter_anchor.filter(not_zero)
        # doc_id_to_norm_anchor = docid_2_token_counter_without0.map(lambda pair: (pair[0],calculate_document_norm(pair, inv_index.doclen, w2df_anchor_dict))).collectAsMap()
        doc_id_to_norm_anchor = docid_2_token_counter_without0.map(lambda pair: (pair[0],np.linalg.norm([(pair[1][term]/ doclen_anchor.get(pair[0],1)) * math.log(n_docs/w2df_anchor_dict.get(term, n_docs),10)for term in pair[1]]))).collectAsMap()
        inv_index.doc_id_2_norm = doc_id_to_norm_anchor
        path_name = 'index_anchor'
        inv_index.write_index('.', 'index_anchor')

    
    # Upload the index to the GCP
    index_source = path_name +".pkl"
    index_dest = f'gs://{bucket_name}/postings_gcp/{path_name}/{index_source}'
    !gsutil cp $index_source $index_dest
    
    return inv_index

In [ ]:
text_inverted_index = build_index_by_part('text')

In [ ]:
title_inverted_index = build_index_by_part('title')

In [ ]:
anchor_inverted_index = build_index_by_part('anchor')